In [3]:
!pip install captcha

  Using cached captcha-0.3-py3-none-any.whl (101 kB)


In [4]:
from captcha.image import ImageCaptcha
from PIL import Image
import random
import numpy as np

CHARSETS = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

In [5]:
class CaptchaGenerator(object):
  
  def __init__(self, image_length):
    self.image_length = image_length
    self.image_captcha = ImageCaptcha()

  def get_random_text(self):
    retVal = ''
    for _ in range(self.image_length):
        retVal += CHARSETS[random.randint(0, len(CHARSETS)-1)]
    return retVal

  def get_labeled_image(self, samples):
    X, Y = [], []
    for _ in range(samples):
        text = self.get_random_text()
        image = self.image_captcha.generate(text, format='png')
        captcha_image = Image.open(image)
        array = np.array(captcha_image)
        X.append(array)
        Y.append(text)
        #self.image_captcha.write(text, "%s.png" % text)
    return X, Y

In [6]:
import os
import os.path
import cv2
import glob
import imutils


CAPTCHA_IMAGE_FOLDER = "generated_captcha_images"
OUTPUT_FOLDER = "extracted_letter_images"

def letters_segmentation(captcha_image_folder, output_folder):
  # Get a list of all the captcha images we need to process
  captcha_image_files = glob.glob(os.path.join(captcha_image_folder, "*"))
  counts = {}


  # loop over the image paths
  for (i, captcha_image_file) in enumerate(captcha_image_files):
      print("[INFO] processing image {}/{}".format(i + 1, len(captcha_image_files)))

      # Since the filename contains the captcha text (i.e. "2A2X.png" has the text "2A2X"),
      # grab the base filename as the text
      filename = os.path.basename(captcha_image_file)
      captcha_correct_text = os.path.splitext(filename)[0]

      # Load the image and convert it to grayscale
      image = cv2.imread(captcha_image_file)
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

      # Add some extra padding around the image
      gray = cv2.copyMakeBorder(gray, 8, 8, 8, 8, cv2.BORDER_REPLICATE)

      # threshold the image (convert it to pure black and white)
      thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

      # find the contours (continuous blobs of pixels) the image
      contours = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

      # Hack for compatibility with different OpenCV versions
      contours = contours[0] 

      letter_image_regions = []

      # Now we can loop through each of the four contours and extract the letter
      # inside of each one
      for contour in contours:
          # Get the rectangle that contains the contour
          (x, y, w, h) = cv2.boundingRect(contour)

          # Compare the width and height of the contour to detect letters that
          # are conjoined into one chunk
          if w / h > 1.25:
              # This contour is too wide to be a single letter!
              # Split it in half into two letter regions!
              half_width = int(w / 2)
              letter_image_regions.append((x, y, half_width, h))
              letter_image_regions.append((x + half_width, y, half_width, h))
          else:
              # This is a normal letter by itself
              letter_image_regions.append((x, y, w, h))

      # If we found more or less than 4 letters in the captcha, our letter extraction
      # didn't work correcly. Skip the image instead of saving bad training data!
      if len(letter_image_regions) != 4:
          continue

      # Sort the detected letter images based on the x coordinate to make sure
      # we are processing them from left-to-right so we match the right image
      # with the right letter
      letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])

      # Save out each letter as a single image
      for letter_bounding_box, letter_text in zip(letter_image_regions, captcha_correct_text):
          # Grab the coordinates of the letter in the image
          x, y, w, h = letter_bounding_box

          # Extract the letter from the original image with a 2-pixel margin around the edge
          letter_image = gray[y - 2:y + h + 2, x - 2:x + w + 2]

          # Get the folder to save the image in
          save_path = os.path.join(output_folder, letter_text)

          # if the output directory does not exist, create it
          if not os.path.exists(save_path):
              os.makedirs(save_path)

          # write the letter image to a file
          count = counts.get(letter_text, 1)
          p = os.path.join(save_path, "{}.png".format(str(count).zfill(6)))
          cv2.imwrite(p, letter_image)

          # increment the count for the current key
          counts[letter_text] = count + 1


In [8]:
!pip install keras

In [7]:
import cv2
import pickle
import os
import os.path
import glob
import imutils
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense

from keras.models import load_model

ModuleNotFoundError: No module named 'keras'

In [9]:
class CaptchaRecog(object):
  
  def __init__(self, captcha_length):
    self.captcha_length = captcha_length
    self.image_height = 60
    self.image_width = 160

    self.captcha_generator = CaptchaGenerator(captcha_length)

  def load_dataset(self, trainset_size, testset_size):
    # Both trainset and dataset will be coming from the same distribution
    self.X_train, self.Y_train = \
      self.captcha_generator.get_labeled_image(trainset_size)
    self.X_test, self.Y_test = \
      self.captcha_generator.get_labeled_image(testset_size)

    # image_arr = self.grey_image(image_arr)
    for i in range(trainset_size):
      self.X_train[i] = self.X_train[i] / 255.
      self.Y_train[i] = self.onehot_encode(self.Y_train[i])
    for i in range(testset_size):
      self.X_test[i] = self.X_test[i] / 255.
      self.Y_test[i] = self.onehot_encode(self.Y_test[i])

    print("Train set size: %d" % len(self.X_train))
    print("Test set size: %d" % len(self.X_test))

  def load_dataset_letters(self, dataset_size = 0):
    
    if (dataset_size != 0):
      self.captcha_generator.get_labeled_image(dataset_size)
    
    CAPTCHA_IMAGE_FOLDER = "generated_captcha_images"
    LETTER_IMAGES_FOLDER = "extracted_letter_images"
    MODEL_LABELS_FILENAME = "model_labels.dat"
    letters_segmentation(CAPTCHA_IMAGE_FOLDER, LETTER_IMAGES_FOLDER)

    # initialize the data and labels
    data = []
    labels = []

    # loop over the input images
    for image_file in paths.list_images(LETTER_IMAGES_FOLDER):
        # Load the image and convert it to grayscale
        image = cv2.imread(image_file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Resize the letter so it fits in a 20x20 pixel box
        image = resize_to_fit(image, 20, 20)

        # Add a third channel dimension to the image to make Keras happy
        image = np.expand_dims(image, axis=2)

        # Grab the name of the letter based on the folder it was in
        label = image_file.split(os.path.sep)[-2]

        # Add the letter image and it's label to our training data
        data.append(image)
        labels.append(label)
    
    # scale the raw pixel intensities to the range [0, 1] (this improves training)
    data = np.array(data, dtype="float") / 255.0
    labels = np.array(labels)

    # Split the training data into separate train and test sets
    (self.X_train, self.X_test, self.Y_train, self.Y_test) = train_test_split(data, labels, test_size=0.25, random_state=0)

    # Convert the labels (letters) into one-hot encodings that Keras can work with
    lb = LabelBinarizer().fit(self.Y_train)
    self.Y_train = lb.transform(self.Y_train)
    self.Y_test = lb.transform(self.Y_test)

    # Save the mapping from labels to one-hot encodings.
    # We'll need this later when we use the model to decode what it's predictions mean
    with open(MODEL_LABELS_FILENAME, "wb") as f:
      pickle.dump(lb, f)

    print("Train set size: %d" % len(self.X_train))
    print("Test set size: %d" % len(self.X_test))


  def read_image(self, img_file):
    label = img_file.split('.')[0]
    image = Image.open(img_file)
    array = np.array(image)

    return label, array
    
  def grey_image(self, img):
    r, g, b = img[:,:,0], img[:,:,1], img[:,:,2]
    grey = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return grey

  def onehot_encode(self, text):
    vector = np.zeros(self.captcha_length * len(CHARSETS))
    for idx, char in enumerate(text):
      vector[idx * len(CHARSETS) + CHARSETS.index(char)] = 1
    return vector

  def onehot_decode(self, vector):
    v = np.nonzero(vector)[0]
    text = ''
    for i in range(self.captcha_length):
      text += CHARSETS[v[i] % len(CHARSETS)]
    return text
    
  def model(self):
    # Build the neural network!
    model = Sequential()

    # First convolutional layer with max pooling
    model.add(Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Second convolutional layer with max pooling
    model.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    # Hidden layer with 500 nodes
    model.add(Flatten())
    model.add(Dense(500, activation="relu"))

    # Output layer with 32 nodes (one for each possible letter/number we predict)
    model.add(Dense(32, activation="softmax"))

    # Ask Keras to build the TensorFlow model behind the scenes
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

  def train(self, num_epochs):
    MODEL_FILENAME = "captcha_model2.hdf5"
    model = self.model()
    model.fit(self.X_train, self.Y_train, validation_data=(self.X_test, self.Y_test), batch_size=32, epochs=num_epochs, verbose=1)
    model.save(MODEL_FILENAME)
     
  def predict(self):
    return

In [10]:
def main():
    cr = CaptchaRecog(captcha_length=4)
    cr.load_dataset_letters()
    cr.train(num_epochs=10)

In [11]:
main()

ValueError: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.